In [1]:
import os
import tensorflow as tf
from gpus import gpu_growth
from models import yolov3
from VOC_loder import VOC_dataset
from yolo_v3_loss import yolo_loss

In [2]:
gpu_growth("11")

Total 1 GPUS


In [3]:
DATASET_PATH = "data/PascalVOC/VOC2012"
LABEL_PATH = "train_data/VOC"
COCO_ANCHORS = [10,13, 16,30, 33,23, 30,61, 62,45, 59,119, 116,90, 156,198, 373,326]
image_size = 416

In [4]:
Dataset = VOC_dataset(DATASET_PATH, LABEL_PATH)

In [5]:
yolov3_model = yolov3.Darknet_yolo(num_class=20)

In [6]:
yolov3_model.load_weights("weights/yolov3.h5",by_name=True, skip_mismatch=True)

In [7]:
optimizer = tf.optimizers.Adam(0.0001)

In [8]:
@tf.function
def train_step(yolov3_model, data, yolo_loss):
    with tf.GradientTape() as tape:
        loss = yolo_loss(yolov3_model, data[0], data[3], data[2], data[1], COCO_ANCHORS,image_size)
    gradients = tape.gradient(loss, yolov3_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, yolov3_model.trainable_variables))
    return loss

In [9]:
i = 0
for data in Dataset:
    loss = train_step(yolov3_model, data, yolo_loss)
    i += 1
    if i % 100 == 0:  
        print(i)
        print(loss.numpy())
        yolov3_model.save_weights("VOC.h5")
    if i == 1000000:
        break

100
5.2436266
200
3.7964535
300
4.826809
400
4.4757433
500
4.80964
600
3.7750883
700
3.554511
800
3.1380324
900
5.6855936
1000
5.4134464
1100
4.7774224
1200
6.15259
1300
4.7728996
1400
4.3283963
1500
4.117132
1600
4.770114
1700
4.7161264
1800
4.2627006
1900
3.3656278
2000
4.754724
2100
3.697879
2200
3.033183
2300
5.1048927
2400
5.591296
2500
4.437459
2600
2.612431
2700
4.6747885
2800
3.3291035
2900
3.5466275
3000
3.8389773
3100
3.1419187
3200
4.137591
3300
2.6132913
3400
3.8813024
3500
3.1814651
3600
2.8999343
3700
3.603004
3800
2.953598
3900
2.7938302
4000
4.7586384
4100
2.2697215
4200
3.0533385
4300
4.4687977
4400
3.9638255
4500
4.3646746
4600
3.3296566
4700
4.4582253
4800
3.674784
4900
3.340066
5000
2.88683
5100
4.0846915
5200
2.9669602
5300
3.5648615
5400
2.2599049
5500
3.8642554
5600
3.5237288
5700
3.0090346
5800
3.412732
5900
3.4968486
6000
2.9571042
6100
3.3087208
6200
3.1891885
6300
3.3031702
6400
2.9581187
6500
2.6343331
6600
3.1444404
6700
4.0367374
6800
3.3547719
6900
3.7091

KeyboardInterrupt: 

In [ ]:
yolov3_model.save_weights("VOC.h5")